# Импорты

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import json

# Чтение исходных данных

In [2]:
DATAPATH = './data/'

In [3]:
train_file = DATAPATH + 'train.json'
test_file = DATAPATH + 'test.json'

In [4]:
sample = pd.read_csv(DATAPATH + 'sample.csv')
sample.head()

,id,sentiment
0,0,neutral
1,1,positive
2,2,negative
3,3,positive
4,4,neutral


In [5]:
def get_df(json_filename):
    with open(json_filename) as json_file:
        temp = json.load(json_file)
        return pd.DataFrame.from_records(temp)

In [6]:
train = get_df(train_file)
train.head()

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative
1,Медики рассказали о состоянии пострадавшего му...,1957,negative
2,"Прошел почти год, как железнодорожным оператор...",1969,negative
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative


In [7]:
test = get_df(test_file)
test.head()

,text,id
0,"Как сообщает пресс-служба акимата Алматы, для ...",0
1,Казахстанские авиакомпании перевозят 250 тысяч...,1
2,На состоявшемся под председательством Касым-Жо...,2
3,В ОАЭ состоялись переговоры между казахстанско...,3
4,12 вагонов грузового поезда сошли с путей в Во...,4


# Предобработка с помощью UDPipe

## Установка нужных пакетов и загрузка модели

In [8]:
!pip install ufal.udpipe

In [9]:
!pip install wget

In [10]:
import wget
import os

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udpipe_filename = udpipe_url.split('/')[-1]

text_url = 'https://rusvectores.org/static/henry_sobolya.txt'
text_filename = text_url.split('/')[-1]

if not os.path.isfile(udpipe_filename):
    print('UDPipe model not found. Downloading...', file=sys.stderr)
    wget.download(udpipe_url)

if not os.path.isfile(text_filename):
    print('{} not found. Downloading...'.format(text_filename), file=sys.stderr)
    wget.download(text_url)

## Тестовые тексты

In [19]:
test_text_0 = 'Моя мама тщательно мыла грязную раму новым мылом до блеска и сверкания.'
test_text_0

'Моя мама тщательно мыла грязную раму новым мылом до блеска и сверкания.'

In [11]:
test_text_1 = train['text'][0]
test_text_1

'Досудебное расследование по факту покупки ЕНПФ пакета облигаций ТОО "Бузгул Аурум" было начато по инициативе Национального банка РК, сообщил директор департамента защиты прав потребителей и финансовых услуг Нацбанка Казахстана Александр Терентьев.\n"Основанием для досудебного расследования стало обращение Национального банка, письмо от 25 ноября 2016 года. Было обращение Национального банка в правоохранительные органы. Нам эта сделка показалась сомнительной, недостаточно корректной, поэтому Нацбанк 25 ноября 2016 года обратился в правоохранительные органы. Это то, что я могу озвучить на сегодня. Идёт следствие, проводится проверка", – сказал Терентьев.\n28 декабря в Нацбанке заявили, что не знают, что стало основанием для проверки ЕНПФ.\n23 декабря факт проведения проверки в АО "Единый накопительный пенсионный фонд" подтвердился. Пресс-служба Национального банка сообщила, что проверку проводят по операциям, совершённым АО "ЕНПФ" в отношении инвестирования собственных активов.\nТакже в

In [12]:
test_text_2 = train['text'][1]
test_text_2

'Медики рассказали о состоянии пострадавшего мужчины, на которого было совершено нападение возле отделения банка по Тимирязева. Как прокомментировали Tengrinews.kz в пресс-службе Управления здравоохранения Алматы, с места происшествия в службу скорой помощи обратились двое человек. \n\n«Одному из них на месте была оказана медицинская помощь. От госпитализации он отказался. Второй пациент был доставлен в больницу скорой неотложной помощи (БСНП) с сотрясением головного мозга, ушибленной раной головы. Состояние на данный момент оценивается ближе к удовлетворительному. Пока он проходит обследование в больнице», — сообщили в Управлении здравоохранения Алматы.  \n\nНапомним, в Алматы на пересечении улиц Тимирязева и Маркова возле БЦ «Алатау Гранд» произошла стрельба, ориентировочно в обеденное время. В здании расположены отделения банков «ВТБ» и «Сбербанк». \n\nВ настоящее время полицейские разыскивают подозреваемых в стрельбе. По факту нападения в местном управлении внутренних дел начато до

In [13]:
test_text_3 = open(text_filename, 'r', encoding='utf-8').read()
test_text_3

'Русские соболя. О. Генри\nКогда синие, как ночь, глаза Молли Мак-Кивер положили Малыша Брэди на обе лопатки, он вынужден был покинуть ряды банды «Дымовая труба». Такова власть нежных укоров подружки и ее упрямого пристрастия к порядочности. Если эти строки прочтет мужчина, пожелаем ему испытать на себе столь же благотворное влияние завтра, не позднее двух часов пополудни, а если они попадутся на глаза женщине, пусть ее любимый шпиц, явившись к ней с утренним приветом, даст пощупать свой холодный нос — залог собачьего здоровья и душевного равновесия хозяйки.\nБанда «Дымовая труба» заимствовала свое название от небольшого квартала, который представляет собой вытянутое в длину, как труба, естественное продолжение небезызвестного городского района, именуемого Адовой кухней. Пролегая вдоль реки, параллельно Одиннадцатой и Двенадцатой авеню, Дымовая труба огибает своим прокопченным коленом маленький, унылый, неприютный Клинтон-парк. Вспомнив, что дымовая труба — предмет, без которого не обх

## Получение Conllu формата

In [23]:
from ufal.udpipe import Model, Pipeline

model = Model.load(udpipe_filename)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
processed = process_pipeline.process(test_text_0)

print(processed)

# newdoc
# newpar
# sent_id = 1
# text = Моя мама тщательно мыла грязную раму новым мылом до блеска и сверкания.
1	Моя	мой	DET	_	Case=Nom|Gender=Fem|Number=Sing	2	det	_	_
2	мама	мама	NOUN	_	Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing	4	nsubj	_	_
3	тщательно	тщательно	ADV	_	Degree=Pos	4	advmod	_	_
4	мыла	мыть	VERB	_	Aspect=Imp|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
5	грязную	грязный	ADJ	_	Case=Acc|Degree=Pos|Gender=Fem|Number=Sing	6	amod	_	_
6	раму	рама	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	4	obj	_	_
7	новым	новый	ADJ	_	Case=Ins|Degree=Pos|Gender=Masc|Number=Sing	8	amod	_	_
8	мылом	мыло	NOUN	_	Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing	6	nmod	_	_
9	до	до	ADP	_	_	10	case	_	_
10	блеска	блеск	NOUN	_	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing	8	nmod	_	_
11	и	и	CCONJ	_	_	12	cc	_	_
12	сверкания	сверкание	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	10	conj	_	SpaceAfter=No
13	.	.	PUNCT	_	_	12	punct	_	SpacesAfter=\n




In [24]:
import unify